In [192]:
import requests
import json
import pandas as pd

In [193]:
address = "0xF1748222B08193273fd34FF10A28352A2C25Adb0".lower()
query = """query {
  account(id: "addressForQuery") {
    inflows(where: {currentFlowRate_gt: "0"}) {
      currentFlowRate
      createdAtTimestamp
      sender {
        id
      }
      token {
        symbol
      }
    }
    outflows(where: {currentFlowRate_gt: "0"}) {
      currentFlowRate
      createdAtTimestamp
      receiver {
        id
      }
      token {
        symbol
      }
    }
  }
}"""

query = query.replace("addressForQuery", address)

In [194]:
url = 'https://api.thegraph.com/subgraphs/name/superfluid-finance/protocol-v1-matic'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

200
{"data":{"account":{"inflows":[{"currentFlowRate":"79000000000000","createdAtTimestamp":"1665526783","sender":{"id":"0x005f16f017aa933bb41965b52848ceb8ee48b171"},"token":{"symbol":"USDCx"}},{"currentFlowRate":"38000000000000","createdAtTimestamp":"1656694205","sender":{"id":"0x0154d25120ed20a516fe43991702e7463c5a6f6e"},"token":{"symbol":"USDCx"}},{"currentFlowRate":"11000000000000","createdAtTimestamp":"1668535555","sender":{"id":"0x169e557af210203a5289beed805b854bf118cbfa"},"token":{"symbol":"USDCx"}},{"currentFlowRate":"19000000000000","createdAtTimestamp":"1665417809","sender":{"id":"0x18b8ce7affb71bfc3ec60aeb249a5f34334951d4"},"token":{"symbol":"USDCx"}},{"currentFlowRate":"3858000000000000","createdAtTimestamp":"1667726552","sender":{"id":"0x1b5d1ada2722413633475fe9745f6041eaaf10ef"},"token":{"symbol":"USDCx"}},{"currentFlowRate":"1000000000000","createdAtTimestamp":"1668336749","sender":{"id":"0x23de5ff9907d465e874fafab51b73a4b8f19945a"},"token":{"symbol":"USDCx"}},{"currentF

In [195]:
json_data = json.loads(r.text)

In [196]:
# Get outflows data
df_data = json_data['data']['account']['outflows']
outflow_df = pd.DataFrame(df_data)

# Getting address value in `receiver` column of dictionaries if there is one
try:
    
    outflow_df['receiver'] = outflow_df['receiver'].apply(lambda x: x['id'])
    outflow_df['token'] = outflow_df['token'].apply(lambda x: x['symbol'])

    # Assigning data types
    outflow_df = outflow_df.astype({
        'currentFlowRate'    : 'int64',
        'createdAtTimestamp' : 'int64',
        'receiver'           : "str",
        'token'              : 'str'
    })

    # Make outflows negative
    outflow_df['currentFlowRate'] = -outflow_df['currentFlowRate']

except:
    pass

print(outflow_df.dtypes)
outflow_df.head()

Series([], dtype: object)


""


In [197]:
# Get inflows data
df_data = json_data['data']['account']['inflows']
inflow_df = pd.DataFrame(df_data)

# Getting address value in `sender` column of dictionaries if there is one
try:

    inflow_df['sender'] = inflow_df['sender'].apply(lambda x: x['id'])
    inflow_df['token'] = inflow_df['token'].apply(lambda x: x['symbol'])
    
    # Assigning data types
    inflow_df = inflow_df.astype({
        'currentFlowRate'    : 'int64',
        'createdAtTimestamp' : 'int64',
        'sender'             : "str",
        'token'              : 'str'
    })
    
except:
    pass

print(inflow_df.dtypes)
inflow_df.head()

currentFlowRate        int64
createdAtTimestamp     int64
sender                object
token                 object
dtype: object


,currentFlowRate,createdAtTimestamp,sender,token
0,79000000000000,1665526783,0x005f16f017aa933bb41965b52848ceb8ee48b171,USDCx
1,38000000000000,1656694205,0x0154d25120ed20a516fe43991702e7463c5a6f6e,USDCx
2,11000000000000,1668535555,0x169e557af210203a5289beed805b854bf118cbfa,USDCx
3,19000000000000,1665417809,0x18b8ce7affb71bfc3ec60aeb249a5f34334951d4,USDCx
4,3858000000000000,1667726552,0x1b5d1ada2722413633475fe9745f6041eaaf10ef,USDCx


In [198]:
# append them
df = pd.concat([inflow_df, outflow_df])

In [199]:
# wei/sec * 1 ether/10**18 wei * (60*60*24*30) sec / 1 month = ethers/month
df_final = df.assign(currentFlowRateMonthly = lambda x : ( x['currentFlowRate'] / (10**18) ) * (60*60*24*30) )[["token","currentFlowRateMonthly","sender"]]

df

KeyError: "['receiver'] not in index"

In [ ]:
df_final.fillna(address)


,token,currentFlowRateMonthly,sender,receiver
0,USDCx,7397.282880,0x290a27026021af22b1794dc1e489a35869820637,0xa852830defa900d655cb933c62474d3c85954fc5
0,USDCx,-2054.794521,0xa852830defa900d655cb933c62474d3c85954fc5,0x4e26f0ded5597bac558cbe472fe6d10de5d0944b
